# Dataset generation
We are going to create a dataset based on your (mine ?) spotify playlist. You can try it yourself by executing this notebook or see the results i've had.

The dataset generated is then saved as CSV at the end of this notebook
## Set up
You can create a spotify app by:
- going to https://developer.spotify.com/dashboard/
- On the dashboard, select the redirect uri to be: http://localhost:3000


### Imports

In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

### Setting up important variables
The library used (spotipy) needs the credentials of a real spotify App created on your dashboard

In [2]:
cid = '1be9accc1f3c4829a2fcc3c757d96bff'
secret = '0c62869c338048378395a751b57270ad'
redirect_uri = "http://localhost:6532"



username = "notebook datascience app"

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# Get read access to your library
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [5]:
# creating the pandas dataframe
df = pd.DataFrame()
df_saved_tracks = pd.DataFrame()

# the index because the song retrieval is done with pagination
offset_index = 0

# is there more songs to processed ?
more_songs = True

track_list = ''
added_ts_list = []
artist_list = []
title_list = []
genre_list = []
principal_genre_list = []


# while there are songs
while more_songs:
    songs = sp.current_user_saved_tracks(offset=offset_index)
    
    for song in songs['items']:
        
        # join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
                # getting the artists
        artists = song['track']['artists']
        artist_details = sp.artist(artists[0]['id'])

        # get the time when the song was added
        added_ts_list.append(song['added_at'])
        
        # get the title of the song
        title_list.append(song['track']['name'])
        
        # getting the artists
        artists = song['track']['artists']
        
        # adding the genre list
        genre_list += [artist_details['genres']]
        principal_genre = ""
        if len(artist_details["genres"]):
            principal_genre = artist_details['genres'][0]
        #print(f"Adding song: {song['track']['name']} - {song['track']['artists'][0]['name']} - Principal genre: {principal_genre}")

        tmp = "other"
        for genre in artist_details['genres']:
            if "techno" in genre:
                tmp = "techno"
                break
            elif "house" in genre:
                tmp = "house"
                break
            elif "dub" in genre:
                tmp = "reggae"
                break
            elif "pop" in genre:
                tmp = "pop"
                break
            elif "classic" in genre:
                tmp = "classic"
                break
            elif "dance" in genre:
                tmp = "dance"
                break
            elif "rap" in genre:
                tmp = "rap"
                break
            elif "disco" in genre:
                tmp = "disco"
                break
            elif "psytrance" in genre:
                tmp = "techno"
                break
            elif "minimale" in genre:
                tmp = "techno"
                break
            elif "minimal" in genre:
                tmp = "techno"
                break
            elif "jazz" in genre:
                tmp = "jazz"
                break
            elif "drum and bass"in genre:
                tmp = "drum and bass"
                break
            elif "jungle"in genre:
                tmp = "drum and bass"
                break
            elif "neurofunk"in genre:
                tmp = "drum and bass"
                break
            
        principal_genre_list.append(tmp) 
        if tmp == "other":
            print(f"other spotted for: {artist_details['genres']}")
        # get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
        
    #get the track features and append into a dataframe
    track_features = sp.audio_features(track_list[:-1])
    
    # adding the tracks details to the current main dataset
    df_temp = pd.DataFrame(track_features)    
    df_saved_tracks = df_saved_tracks.append(df_temp)
    track_list = ''
    
    # check if there are more songs to be processed
    if songs['next'] == None:
        # no more songs in playlist
        more_songs = False
    else:
        # get the next n songs
        offset_index += songs['limit']
#include timestamp added, title and artists of a song
df_saved_tracks['added_at'] = added_ts_list
df_saved_tracks['song_title'] = title_list
df_saved_tracks['artists'] = artist_list
df_saved_tracks['genres'] = genre_list
df_saved_tracks['principal_genre'] = principal_genre_list

df_saved_tracks

other spotted for: []
other spotted for: ['french indietronica', 'new french touch']
other spotted for: ['chanson', 'french rock']
other spotted for: ['french indietronica', 'new french touch']
other spotted for: ['french indietronica']
other spotted for: ['new french touch']
other spotted for: ['downtempo', 'electronica']
other spotted for: ['deep liquid bass', 'liquid funk', 'uk dnb']
other spotted for: []
other spotted for: []
other spotted for: ['lithuanian electronic']
other spotted for: []
other spotted for: ['australian reggae fusion', 'cuban alternative']
other spotted for: []
other spotted for: ['lithuanian electronic']
other spotted for: ['alternative rock', 'anti-folk', 'garage rock', 'modern rock', 'permanent wave', 'rock']
other spotted for: ['retro electro']
other spotted for: ['downtempo', 'electronica']
other spotted for: ['chill beats']
other spotted for: []
other spotted for: ['electronica', 'future garage', 'indie soul']
other spotted for: ['modern reggae']
other spo

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,added_at,song_title,artists,genres,principal_genre
0,0.661,0.765,2,-8.818,1,0.0297,0.0757,0.8010,0.0919,0.2370,...,spotify:track:38CngZw3TDi2XDCQPMKHJR,https://api.spotify.com/v1/tracks/38CngZw3TDi2...,https://api.spotify.com/v1/audio-analysis/38Cn...,228293,4,2020-04-07T16:07:14Z,A Sheltered Place - Phaeleh Remix,"Way Out West,Phaeleh","[classic progressive house, nu skool breaks, p...",house
1,0.728,0.784,8,-12.477,1,0.0414,0.1030,0.8480,0.0858,0.0646,...,spotify:track:6YJ9z4IInxfJWsjc6ADtV2,https://api.spotify.com/v1/tracks/6YJ9z4IInxfJ...,https://api.spotify.com/v1/audio-analysis/6YJ9...,377919,3,2020-04-07T13:40:07Z,Prelusion,"Mark Tarmonea,Yannek Maunz",[german house],house
2,0.725,0.742,0,-8.646,1,0.0328,0.0887,0.8100,0.1070,0.0980,...,spotify:track:4isQapTCsf1TvF9RkzfrCG,https://api.spotify.com/v1/tracks/4isQapTCsf1T...,https://api.spotify.com/v1/audio-analysis/4isQ...,277073,4,2020-04-07T09:13:47Z,Odyssee - Pt. 2,BLR,"[dutch edm, house]",house
3,0.689,0.810,8,-6.621,0,0.0464,0.0203,0.9010,0.1390,0.8630,...,spotify:track:0oB0mWnbRYNsC2CicOkXZL,https://api.spotify.com/v1/tracks/0oB0mWnbRYNs...,https://api.spotify.com/v1/audio-analysis/0oB0...,407803,4,2020-04-06T14:33:04Z,GUFM - Folamour Remix,"Mangabey,Folamour",[],other
4,0.946,0.493,8,-9.654,1,0.0509,0.1620,0.2970,0.0875,0.5920,...,spotify:track:0aKoR2iMGuqAUBBCdRwII8,https://api.spotify.com/v1/tracks/0aKoR2iMGuqA...,https://api.spotify.com/v1/audio-analysis/0aKo...,270000,4,2020-04-04T12:08:27Z,Prettiest Virgin (Radio Edit),Agar Agar,"[electronica, french indie pop, french indietr...",pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.692,0.626,11,-11.361,0,0.0637,0.0432,0.7970,0.1100,0.5360,...,spotify:track:4bkTaAK3tvBi7M9S6UEyy2,https://api.spotify.com/v1/tracks/4bkTaAK3tvBi...,https://api.spotify.com/v1/audio-analysis/4bkT...,353541,4,2016-12-28T16:20:11Z,StillDreaming,Hypnagog,"[ambient psychill, deep psytrance]",techno
2,0.489,0.571,10,-11.268,0,0.0933,0.1480,0.6660,0.1830,0.4910,...,spotify:track:6H0HOzCvYYbzdhhrOVBzxb,https://api.spotify.com/v1/tracks/6H0HOzCvYYbz...,https://api.spotify.com/v1/audio-analysis/6H0H...,262139,1,2016-12-28T16:20:11Z,FunkDiddle,Hypnagog,"[ambient psychill, deep psytrance]",techno
3,0.700,0.581,6,-7.116,1,0.1500,0.3580,0.0539,0.0825,0.6330,...,spotify:track:3ndu6BpfkiGE4m0BboNUHv,https://api.spotify.com/v1/tracks/3ndu6BpfkiGE...,https://api.spotify.com/v1/audio-analysis/3ndu...,277385,4,2016-12-28T14:29:04Z,Tempo,"Ondubground,Biga Ranx","[dub reggae, electro dub, french dub]",reggae
4,0.638,0.724,6,-7.291,1,0.0270,0.0206,0.5250,0.1390,0.3500,...,spotify:track:5MFtsrwzB0SoZzp2QvaSmF,https://api.spotify.com/v1/tracks/5MFtsrwzB0So...,https://api.spotify.com/v1/audio-analysis/5MFt...,243016,4,2016-12-28T14:28:35Z,Because,Ondubground,"[dub reggae, electro dub, french dub]",reggae


In [14]:
# cheking the types
df_saved_tracks.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
added_at             object
song_title           object
artists              object
genres               object
principal_genre      object
dtype: object

## save the dataset
Saving as csv for further use

In [15]:
df_saved_tracks.to_csv("./datasets/playlist-gabriel.csv")